## Imagery layers and Raster analysis


In [ ]:
import arcgis

### Access imagery

In [ ]:
from arcgis.gis import GIS

In [ ]:
gis = GIS("https://slustl.maps.arcgis.com/", "gregbrunner_slugis")

In [ ]:
landsat_item = gis.content.search('title: Multispectral Landsat',
                                  'Imagery Layer', outside_org=True)[0]

In [ ]:
landsat_item.layers

### View Landsat imagery layer item description

In [ ]:
from IPython.display import HTML
HTML(landsat_item.description)

### Access the layers available with the Landsat Imagery Layer item

In [ ]:
landsat = landsat_item.layers[0]
landsat 

## Explore different wavelength bands

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(landsat.key_properties()['BandProperties'])

## Visualize the layer in the map widget

In [ ]:
m = gis.map('Paris, France')
m

In [ ]:
m.add_layer(landsat)

In [ ]:
import time
from arcgis.raster.functions import apply

for idx, rasterfunc in enumerate(landsat.properties.rasterFunctionInfos):
    m.add_layer(apply(landsat, rasterfunc.name))
    print(rasterfunc.name)
    time.sleep(2)
    if idx > 4:
        break

### Interactive raster processing in Jupyter Notebook

In [ ]:
color_infrared = apply(landsat, 'Color Infrared with DRA')

In [ ]:
color_infrared 

In [ ]:
m = gis.map('Laval, France')
m

In [ ]:
m.add_layer(color_infrared)

### Setting an area of interest

In [ ]:
from arcgis.geocoding import geocode
area = geocode('Paris, France', out_sr=landsat.properties.spatialReference)[0]

In [ ]:
color_infrared.extent = area['extent']

In [ ]:
color_infrared

In [ ]:
landsat.extent = area['extent']
landsat

## Exporting Images from Imagery Layer

In [ ]:
from IPython.display import Image

### Exports an image to binary

In [ ]:
img = landsat.export_image(bbox=area['extent'], size=[1200,450], f='image')

In [ ]:
Image(img)

In [ ]:
savedimg = landsat.export_image(bbox=area['extent'], size=[1200,450], 
                                f='image', save_folder='.', 
                                save_file='img.jpg')

In [ ]:
savedimg

In [ ]:
from IPython.display import Image

In [ ]:
Image(filename=savedimg, width=1200, height=450)

### Exporting images with raster function applied to them

In [ ]:
ndvi_colorized = apply(landsat, 'NDVI Colorized')
ndvi_colorized.extent = area['extent']
ndvi_colorized

## Extracting custom bands

In [ ]:
from arcgis.raster.functions import stretch, extract_band

In [ ]:
naturalcolor = stretch(extract_band(landsat, [3,2,1]), 
                    stretch_type='percentclip', min_percent=0.1, max_percent=0.1, 
                       gamma=[1, 1, 1], dra=True)
naturalcolor.extent = area['extent']
naturalcolor

# Clipping to an area of interest

In [ ]:
from arcgis.geometry import Geometry, buffer 
from arcgis.raster.functions import clip, apply
%config IPCompleter.greedy=True

In [ ]:
poly = buffer(geometries=[Geometry(area['location'])],
              in_sr=102100, distances=6000, unit=9001)[0]

In [ ]:
from  arcgis.geoenrichment import Country
usa = Country.get('US')
redlands = usa.subgeographies.states['California'].zip5['92373']

In [ ]:
redclip = clip(apply(landsat, 'NDVI Colorized'), redlands.geometry)

In [ ]:
m = gis.map('Redlands, CA')

In [ ]:
m 

In [ ]:
m.add_layer(redclip)

## Select images by where clause, geometry and time range

In [ ]:
import arcgis

In [ ]:
sentinel_item = gis.content.search('Sentinel-2 Views', outside_org=True)[0]
sentinel_item

In [ ]:
sentinel = sentinel_item.layers[0]
sentinel.extent = area['extent']

In [ ]:
selected = sentinel.filter_by(where="cloudcover <= .01 and category = 1",
                              geometry=arcgis.geometry.filters.intersects(area['extent']))

df = selected.query(out_fields="AcquisitionDate, Tile_ID, CloudCover", order_by_fields="AcquisitionDate").sdf
df['acquisitiondate'] = pd.to_datetime(df['acquisitiondate'], unit='ms')
df.tail()

Looking at the shape of the dataframe we see that multiple scenes match the specified criteria:

In [ ]:
df.shape

The footprints of the rasters matching the criteria can be drawn using the map widget:

In [ ]:
df['Time'] = pd.to_datetime(df['acquisitiondate'], unit='ms')
df['Time'].tail(10)

# Change Detection

In [ ]:
new = selected.last()
new

In [ ]:
old = selected.first()
old

## Difference Image

Difference Image mode illustrates all the changes in NDVI (vegeration index) between the two dates:

increases are shown in green, and decreases are shown in magenta. 

In [ ]:
from arcgis.raster.functions import *

In [ ]:
diff = stretch(composite_band([ndvi(old, '3 4'),
                               ndvi(new, '3 4')]), 
                               stretch_type='stddev',  num_stddev=3, min=0, 
                               max=255, dra=True, astype='u8')
diff

In [ ]:
dmap = gis.map("Paris, France")
dmap.add_layer(diff)
dmap

### Persisting your analysis for visualizaion or analysis

```python
lyr = diff.save('Test_viz_layer3')
```

In [ ]:
lyr = diff.save("sent_lyr_diff", for_viz = True)
lyr